In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s51.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
51-05748,0,0,8,8,8,99.0,84,0.0,77,120.0,...,0,0,0,0,1,0,40,24,2,1
51-15063,0,0,8,8,8,99.0,74,70.0,1,124.4,...,0,0,0,0,0,0,25,28,1,1
51-14676,0,0,8,8,8,99.0,74,70.0,1,124.4,...,0,0,0,0,0,0,33,23,1,1
51-03203,0,0,8,8,8,99.0,74,0.0,77,124.4,...,0,0,0,0,0,0,34,18,1,0
51-08914,0,0,8,8,8,2.5,80,60.0,1,130.0,...,0,0,0,0,1,0,40,27,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51-03948,0,0,8,8,8,99.0,74,100.0,77,124.4,...,0,0,0,0,0,0,43,21,2,1
51-00031,0,0,8,8,8,99.0,74,100.0,1,124.4,...,0,0,0,0,0,0,23,17,1,0
51-07760,0,0,8,8,8,99.0,74,80.0,1,124.4,...,0,0,0,0,0,0,32,21,1,1


In [3]:
X.shape

(14959, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(10471, 195)
(14959, 195)
(14959,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(200, 195)
(200,)
[100 100]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[3012 1433]
 [  11   32]]
              precision    recall  f1-score   support

           0       1.00      0.68      0.81      4445
           1       0.02      0.74      0.04        43

    accuracy                           0.68      4488
   macro avg       0.51      0.71      0.42      4488
weighted avg       0.99      0.68      0.80      4488

ROC AUC prob1: 0.7528631595469172
MCC: 0.08762745214593817
F2: 0.09773976786805132
PR_AUC: 0.03476844401727625
CPU times: user 170 ms, sys: 6.98 ms, tot

In [34]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier(random_state=7, verbose=1, n_estimators=400, learning_rate=.1)  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

      Iter       Train Loss   Remaining Time 
         1           1.3170           12.95s
         2           1.2613           11.67s
         3           1.2163           11.22s
         4           1.1764           10.97s
         5           1.1439           10.72s
         6           1.1145           10.57s
         7           1.0868           10.50s
         8           1.0655           10.43s
         9           1.0451           10.43s
        10           1.0255           10.39s
        20           0.9123           10.11s
        30           0.8403           10.01s
        40           0.7902            9.73s
        50           0.7511            9.50s
        60           0.7133            9.21s
        70           0.6761            8.97s
        80           0.6462            8.72s
        90           0.6191            8.42s
       100           0.5940            8.18s
       200           0.4049            5.43s
       300           0.2923            2.69s
       40

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 16:06:29,566] A new study created in memory with name: no-name-cd99b30b-1a1e-4915-9e3a-3d52715ac898
[I 2021-12-10 16:06:30,023] Trial 0 finished with value: 0.09877687612773779 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.09877687612773779.


MCC: [0.09345554365545881, 0.1207972495510677, 0.08013528913425619, 0.10071942217016845]
ROC: [0.7621265298776098, 0.7688909887209023, 0.6829103671706264, 0.7392008639308856]
PR_AUC: [0.023715263948287393, 0.02532114163205567, 0.02033336421774882, 0.031037953921590505]


[I 2021-12-10 16:06:30,749] Trial 1 finished with value: 0.09603059469416647 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.09877687612773779.


MCC: [0.10006831162634375, 0.11304472779599739, 0.06912057299161206, 0.1018887663627127]
ROC: [0.7436255099592032, 0.7751417386609071, 0.6605246580273578, 0.7558238198087011]
PR_AUC: [0.024544059094195114, 0.026369186640705183, 0.021021157958250835, 0.025903559418023074]


[I 2021-12-10 16:06:31,419] Trial 2 finished with value: 0.08878718647504173 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.09877687612773779.


...

MCC: [0.10690747351253803, 0.12088095491280633, 0.09387281593065631, 0.10522888647490955]
ROC: [0.7784902207823374, 0.76532502399808, 0.7169613930885529, 0.7655931811169392]
PR_AUC: [0.03105470592022238, 0.02242797904477649, 0.02555994587172878, 0.0652852567622307]


[I 2021-12-10 16:06:45,255] Trial 23 finished with value: 0.10615605175328258 and parameters: {'n_estimators': 46, 'subsample': 0.8608267291762682, 'max_depth': 20, 'max_features': 15, 'learning_rate': 0.11482662289828563, 'min_samples_leaf': 49}. Best is trial 5 with value: 0.10711949820396768.


MCC: [0.10842127921860473, 0.12146924220577644, 0.08792042836192847, 0.10681325722682064]
ROC: [0.7560632649388048, 0.7742118130549556, 0.7039874310055195, 0.7555808392471459]
PR_AUC: [0.03430521099807913, 0.025816201847752282, 0.02200666887287144, 0.056267795219155904]


[I 2021-12-10 16:06:45,669] Trial 24 finished with value: 0.10948886104872017 and parameters: {'n_estimators': 28, 'subsample': 0.9290306649374853, 'max_depth': 14, 'max_features': 16, 'learning_rate': 0.12914925927473922, 'min_samples_leaf': 35}. Best is trial 24 with value: 0.10948886104872017.


MCC: [0.11378866086505647, 0.12248744297291562, 0.0894117676464688, 0.1122675727104398]
ROC: [0.7707133429325654, 0.7726444384449244, 0.7065522258219343, 0.7630013884603517]
PR_AUC: [0.026935398775932668, 0.024248432368101733, 0.022465231570272613, 0.03970435209421735]
CPU times: user 15.9 s, sys: 325 ms, total: 16.2 s
Wall time: 16.1 s


In [14]:
best = study.best_trial
best

FrozenTrial(number=24, values=[0.10948886104872017], datetime_start=datetime.datetime(2021, 12, 10, 16, 6, 45, 256743), datetime_complete=datetime.datetime(2021, 12, 10, 16, 6, 45, 669113), params={'n_estimators': 28, 'subsample': 0.9290306649374853, 'max_depth': 14, 'max_features': 16, 'learning_rate': 0.12914925927473922, 'min_samples_leaf': 35}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=24, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[3052 1393]
 [  10   33]]
              precision    recall  f1-score   support

           0       1.00      0.69      0.81      4445
           1       0.02      0.77      0.04        43

    accuracy                           0.69      4488
   macro avg       0.51      0.73      0.43      4488
weighted avg       0.99      0.69      0.81      4488

ROC AUC prob1: 0.7417767546498548
MCC: 0.09499860011710942
F2: 0.1032540675844806
PR_AUC: 0.024459407859954145
CPU times: user 66.9 ms, sys: 3.58 ms, total: 70.5 ms
Wall time: 68.2 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[4236  209]
 [  35    8]]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      4445
           1       0.04      0.19      0.06        43

    accuracy                           0.95      4488
   macro avg       0.51      0.57      0.52      4488
weighted avg       0.98      0.95      0.96      4488

ROC AUC prob1: 0.6980249561828027
MCC: 0.06313581986103019
F2: 0.10282776349614395
PR_AUC: 0.03747984192857726
CPU times: user 510 ms, sys: 4.99 ms, total: 515 ms
Wall time: 512 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode             0.088513
new_age             0.074347
BESTGA              0.064599
Admefface           0.063229
new_BMI             0.045447
Lac_Min             0.039832
CS_FTP              0.036352
Dilat_lst           0.031011
Lac_None            0.028408
Delfetalpos         0.026131
momrace_new         0.025598
Admpresent          0.025554
Admreason           0.022888
Analgesia           0.022414
Hxpreterm           0.018354
high_Gravidity      0.018100
ROMmeth             0.016925
Antehospital        0.013785
Malpresentation9    0.013620
Insurance           0.013303
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode           0.258989
CS_FTP            0.122801
Dilat_lst         0.089096
Delfetalpos       0.081375
Admefface         0.073909
spontlabor        0.052047
new_BMI           0.042307
Lac_Min           0.034317
momrace_new       0.033286
BESTGA            0.032297
Insurance         0.030733
Parity            0.024060
MthInd_Oxy        0.023034
Lac_None          0.021233
uscar             0.019975
Induction         0.019037
high_Gravidity    0.009494
Admpresent        0.009227
Ind_Unkn          0.007132
prelaborCD        0.004863
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 28,
 'subsample': 0.9290306649374853,
 'max_depth': 14,
 'max_features': 16,
 'learning_rate': 0.12914925927473922,
 'min_samples_leaf': 35,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>